In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

read data

In [3]:
path_to_file='data.txt'

In [4]:
text=open(path_to_file,'r').read()

In [5]:
#print(text[140500:141500])

unique chars in text

In [6]:
vocab=sorted(set(text))

In [7]:
len(vocab)

84

## Text processing

Assign index to each unique character - for creation of dictionary

In [8]:
char_to_ind={char:ind for ind,char in enumerate(vocab)}

In [9]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

Encode all unique text into integer.

In [10]:
ind_to_char=np.array(vocab)

In [11]:
encoded_text=np.array([char_to_ind[c] for c in text])
    

## Creating Batches

In [12]:
seq_len=120

In [13]:
total_num_seq=len(text)//(seq_len+1)

In [14]:
total_num_seq

45005

create the training sequences

In [15]:
char_dataset=tf.data.Dataset.from_tensor_slices(encoded_text)

In [16]:
sequences=char_dataset.batch(seq_len+1,drop_remainder=True)

In [17]:
def create_seq_targets(seq):
    input_txt=seq[:-1]
    target_txt=seq[1:]
    return input_txt,target_txt

map to all sequences

In [18]:
dataset=sequences.map(create_seq_targets)

In [19]:
dataset

<MapDataset shapes: ((120,), (120,)), types: (tf.int64, tf.int64)>

In [20]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [21]:
batch_size=128

In [22]:
buffer_size=10000

In [23]:
dataset=dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [24]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Creating the model

In [25]:
vocab_size=len(vocab)

In [26]:
vocab_size

84

In [27]:
embed_dim=64

In [28]:
rnn_neurons=1026

Loss function

In [29]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [30]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense

In [32]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    model=Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    
    model.compile('adam',loss=sparse_cat_loss)
    return model

In [33]:
model=create_model(vocab_size=vocab_size,embed_dim=embed_dim,rnn_neurons=rnn_neurons,batch_size=batch_size)

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Training the model

In [36]:
for input_example_batch, target_example_batch in dataset.take(1):
 
 example_batch_predictions=model(input_example_batch)

In [39]:
sampled_indices=tf.random.categorical(example_batch_predictions[0],num_samples=1)

In [41]:
sampled_indices=tf.squeeze(sampled_indices,axis=1).numpy()
sampled_indices

array([49,  8, 25, 42,  2, 61, 51, 44, 58, 33, 73, 18, 17, 17, 12, 17, 44,
       46, 64, 83,  2, 40, 79, 46, 71, 48,  9, 10, 47, 76, 23, 70,  6, 26,
       35, 25, 25, 13, 67, 28, 63, 20, 43, 67, 14, 69,  9, 17, 37,  5, 53,
       28, 15,  2, 43, 58, 31, 60, 23,  9, 20, 63, 65, 65, 61, 44, 73, 24,
       63, 46, 45,  5, 19,  2, 31, 79, 13, 82, 64, 10, 38, 76, 66, 36, 29,
       65,  7, 38, 34, 22, 59, 44, 34, 72, 34, 56, 49, 36, 62, 52, 77, 78,
       40, 58, 19, 35, 23, 28, 47,  4, 25, 28, 81, 26, 55, 64, 48, 44, 46,
       46])

In [42]:
ind_to_char[sampled_indices]

array(['X', ',', '?', 'Q', '!', 'f', 'Z', 'S', 'c', 'H', 'r', '7', '6',
       '6', '1', '6', 'S', 'U', 'i', '}', '!', 'O', 'x', 'U', 'p', 'W',
       '-', '.', 'V', 'u', '<', 'o', '(', 'A', 'J', '?', '?', '2', 'l',
       'C', 'h', '9', 'R', 'l', '3', 'n', '-', '6', 'L', "'", ']', 'C',
       '4', '!', 'R', 'c', 'F', 'e', '<', '-', '9', 'h', 'j', 'j', 'f',
       'S', 'r', '>', 'h', 'U', 'T', "'", '8', '!', 'F', 'x', '2', '|',
       'i', '.', 'M', 'u', 'k', 'K', 'D', 'j', ')', 'M', 'I', ';', 'd',
       'S', 'I', 'q', 'I', 'a', 'X', 'K', 'g', '[', 'v', 'w', 'O', 'c',
       '8', 'J', '<', 'C', 'V', '&', '?', 'C', 'z', 'A', '`', 'i', 'W',
       'S', 'U', 'U'], dtype='<U1')

In [43]:
#model.fit(dataset,epochs=30)

In [44]:
from tensorflow.keras.models import load_model

Generating text

In [45]:
model=create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)
model.load_weights('data.h5')
model.build(tf.TensorShape([1,None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [46]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):
  num_generate=gen_size
  input_eval=[char_to_ind[s] for s in start_seed]
  input_eval=tf.expand_dims(input_eval,0)
  text_generated=[]
  temperature=temp
  model.reset_states()
  for i in range(num_generate):
    predictions =model(input_eval)
    predictions=tf.squeeze(predictions,0)
    predictions=predictions/temperature
    predicted_id= tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
    input_eval=tf.expand_dims([predicted_id],0)
    text_generated.append(ind_to_char[predicted_id])
    return (start_seed+"".join(text_generated))

In [48]:
print(generate_text(model,"JULIET",gen_size=1000))

JULIETH
